https://ofai.github.io/million-post-corpus/ <br> **Posts** - Headline+Body, ID_User

In [1]:
ARTICLES_MIN_LENGTH = 300
ARTICLES_PER_AUTHOR = 250

- fetch Table Posts
- get columns ID_User and Beitrag (Headline + Body)
- remove newline characters
- only get posts with a minimum length of 300
- ignore duplicates

| columns | short posts |
| :-- | :-- |
| ![beitrag](img/headline_body.png)| ![too short post](img/short_body.png) |

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import sqlalchemy as sqla

con = sqlite3.connect("../../OneMillion/million_post_corpus/corpus.sqlite3", isolation_level=None,
                       detect_types=sqlite3.PARSE_COLNAMES)

df = pd.read_sql_query("SELECT distinct ID_User, COALESCE(Headline, '') || COALESCE(Body, '') as Beitrag from Posts", con)

df = df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)

df_minLength = df.loc[df['Beitrag'].apply(len) > ARTICLES_MIN_LENGTH]

In [3]:
## Anzahl Beiträge:
print(df_minLength.count())

ID_User    242143
Beitrag    242143
dtype: int64


In [4]:
## Anzahl der User
print(sum(df_minLength['ID_User'].value_counts()))

242143


In [5]:
## User mit nur einem Beitrag
print(sum(df_minLength['ID_User'].value_counts() == 1))

6274


In [6]:
## User mit mehr als 250 Beiträgen
print(sum(df_minLength['ID_User'].value_counts() > ARTICLES_PER_AUTHOR))

55


In [7]:
## neuer Datensatz ohne User weniger als 251 Beiträge
df_minArticles = df_minLength.groupby('ID_User').filter(lambda x : len(x) > ARTICLES_PER_AUTHOR)

In [8]:
df_minArticles

,ID_User,Beitrag
592,28526,Im großen war es sehr beeindrucktend. Man kann...
605,28526,Mir bleibt ein sehr junger Mann (etwa so um di...
787,6498,ich habe die 1980er damit verbracht in den Ost...
1121,6247,"Vor etwas mehr vielen vielen Jahren, so gegen ..."
1240,6247,Ein Sternehotel im Montafon.In den späten 80ig...
...,...,...
1009952,29722,"Hier wird etwas getan, was bei Gender auch ger..."
1009996,29722,"Andersdenkende einfach als ""Pigs"" also Schwein..."
1009998,29722,"Das ist eine lupenreine Verschwörungstheorie:""..."
1010126,29722,"Sie machen jetzt genau das, was ich im unteren..."


In [9]:
## ID_User - Anzahl passende Beiträge
df_minArticles['ID_User'].value_counts()

5854     729
25987    619
22876    598
30344    574
14587    509
26866    479
28816    456
24984    416
3788     409
20349    408
20141    408
1795     394
18083    385
18446    379
19915    378
15686    376
24667    369
6247     365
8765     363
15886    361
6498     355
10627    352
16722    344
3105     338
24758    337
27393    330
12471    329
30661    320
29722    319
11656    317
26532    312
2690     312
1330     304
21150    304
11910    299
18984    298
18940    297
24343    297
11582    296
12887    288
8446     285
25582    284
29377    284
12071    281
8323     279
1560     275
12023    274
6169     266
18006    264
27991    263
16328    258
7439     255
28526    254
28120    253
28084    252
Name: ID_User, dtype: int64

In [10]:
import random

UserIDListe = df_minArticles.drop_duplicates(subset = ["ID_User"])['ID_User'].tolist()

ARTICLES_PER_AUTHOR_BSP = 250
AUTHORS = 20
IDS = random.sample(UserIDListe, AUTHORS)

In [11]:
## 20 Autoren aus obiger Liste
print(*IDS, sep=', ')

1795, 12887, 27991, 25987, 12071, 18446, 16328, 11910, 26866, 3105, 24343, 28084, 19915, 3788, 25582, 29722, 6498, 12471, 18083, 18006


In [12]:
import os  
os.makedirs('data/users', exist_ok=True)  


In [13]:
for x in range(AUTHORS):
    df_author = df_minArticles[df_minArticles['ID_User'] == IDS[x]].iloc[:ARTICLES_PER_AUTHOR,:]
    df_author.to_csv('data/users/' + str(IDS[x]) + '_posts.csv', index=False, columns=['Beitrag'])